In [3]:
import csv 
import random 
import pandas as pd
import math
import operator
def loadDataset(filename,split,trainingSet = [], testingSet = []):
    with open(filename,"r") as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset) - 1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testingSet.append(dataset[x])
        return dataset,trainingSet,testingSet
    
with open("iris.data.txt") as csvfile:
    lines = csv.reader(csvfile)
    d,t,tt = loadDataset("iris.data.txt",0.5)
    data = pd.DataFrame(d)
print(data)

       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9    3  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4      5  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7    3  5.2  2.3  Iris-virginica
146  6.3  2.5    5  1.9  Iris-virginica
147  6.5    3  5.2    2  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]


In [7]:
def distance(instance1,instance2, method,length,p = 1):
    x = 0
    for i in range(length): #length is equal to len(instance1[0]) - 1  
        if(method == "minkowski"):
            x += ((abs(instance1[i] - instance2[i]))**p)**1/p
        elif(method == "manhattan"):
            x += abs(instance1[i] - instance2[i])
        else: #We select the euclidean distance method as the default one (We can do it through parameters)
            x += math.sqrt((instance1[i] - instance2[i])**2)

    return x

data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distc = distance(data1, data2,"euclidean",3)

print("Distance : " + repr(distc))


Distance : 6.0


In [10]:
def getNeighbors(trainingSet, testInstance,method, k = 1,p = 1):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = distance(testInstance, trainingSet[x], method,length,p)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance,"manhattan")
print(neighbors)

[[4, 4, 4, 'b']]


In [11]:
def getResponse(neighbors):
    classVotes = {}
    for i in range(len(neighbors)):
        response = neighbors[i][len(neighbors[i]) - 1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 0
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    if(len(sortedVotes) >= 2):
        if(sortedVotes[0][1] == sortedVotes[1][1]):
                pass
        else:
            return sortedVotes[0][0]
    else:
        return sortedVotes[0][0]

neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


In [12]:
def getAccuracy(testSet,predictions):
    score = 0
    for i in range(len(predictions)):
        if(predictions[i] in testSet[i]):
            score += 1
    return (score/float(len(testSet))) * 100.0

testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [13]:
def main(filename,split,k,method,p = 1):
    dictionnaire_rps = {}
    dataset,trainingSet,testingSet = loadDataset(filename,split,[],[])
    dataset = pd.DataFrame(dataset)
    for instance in testingSet:
        neighbors = getNeighbors(trainingSet,instance,method,k,p)
        dictionnaire_rps[tuple(instance)] = getResponse(neighbors)
    print("Accuracy  : ", getAccuracy(testingSet,list(dictionnaire_rps.values())))
    df = pd.DataFrame()
    df["Predictions"] = list(dictionnaire_rps.values()) 
    return df #If the value of the predictions is None then it means that the percentages in the majority vote were equal
    
main("iris.data.txt",0.4,2,"minkowski",3)

Accuracy  :  90.58823529411765


,Predictions
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa
...,...
80,None
81,Iris-virginica
82,Iris-versicolor
83,Iris-virginica
